## Create temporaray csv file for the entire year of coverage

This notebook should be the first one running. It takes all the files in `input_data` and combines them in a csv that spans through 1 year of data. The data used as source should have the same structure in order to be combined.

In [ ]:
import os
import json
import pandas as pd
import ast

from dotenv import load_dotenv
from sqlalchemy import create_engine

## NYT Data

In [ ]:
# Define the path to the input-data folder
input_data_folder = '../input-data/nyt'

# List all files in the input-data folder
files = os.listdir(input_data_folder)

# Initialize an empty list to store dataframes
dataframes = []

# Loop through each file and read it into a dataframe
for file in files:
    file_path = os.path.join(input_data_folder, file)
    df = pd.read_json(file_path)
    dataframes.append(df)

# Concatenate all dataframes into one unique dataframe
full_year_nyt = pd.concat(dataframes, ignore_index=True)

# Display the unique dataframe
full_year_nyt

## Zeit Data

In [ ]:
# Define the path to the input-data folder
input_data_folder = '../input-data/zeit'

# List all files in the input-data folder
files = os.listdir(input_data_folder)

# Initialize an empty list to store dataframes
dataframes = []

# Loop through each file and read it into a dataframe
for file in files:
    file_path = os.path.join(input_data_folder, file)
    df = pd.read_json(file_path)
    dataframes.append(df)

# Concatenate all dataframes into one unique dataframe
full_year_zeit = pd.concat(dataframes, ignore_index=True)

# Display the unique dataframe
full_year_zeit

In [ ]:
full_year_zeit["body"] = full_year_zeit["body"].apply(lambda x: x.replace("\n", ""))

## Match Zeit and NYT columns

The data do not match. This section creates the same column structure for both datasets. It can be skipped if different datasets are needed.
🚨: the structure of specific rows remain unchanged.

In [ ]:
full_year_nyt.columns

In [ ]:
full_year_zeit.columns

Mapping of NYT -> Zeit columns, `-` columns will be dropped.
- abstract -> -
- web_url -> url
- snippet -> body
- lead_paragraph -> -
- print_section -> -
- print_page -> -
- source -> source
- multimedia -> image
- headline -> title
- keywords -> keywords
- pub_date -> dateTimePub
- document_type -> dataType 
- news_desk -> -
- section_name -> - 
- byline -> authors
- type_of_material -> -
- _id -> uri
- word_count -> -
- uri -> uri
- subsection_name -> -

Remove columns that cannot be matched.

In [ ]:
full_year_nyt = full_year_nyt.drop(columns=[
    'abstract',
    'lead_paragraph', 
    'print_section', 
    'print_page', 
    'news_desk', 
    'section_name', 
    'type_of_material', 
    'word_count', 
    'subsection_name', 
    'uri'
])

In [ ]:
full_year_zeit = full_year_zeit.drop(columns=[
    'lang', 
    'isDuplicate', 
    'date', 
    'time', 
    'dateTimePub', 
    'sim', 
    'eventUri', 
    'sentiment', 
    'wgt', 
    'relevance',
    'links'
    ])

Rename columns to match datasets.

In [ ]:
full_year_zeit = full_year_zeit.rename(columns={
    'url': 'web_url', 
    'body': 'snippet', 
    'image': 'multimedia', 
    'title': 'headline',
    'dateTime': 'pub_date',
    'dataType': 'document_type',
    'uri': '_id',
    'authors': 'byline'
})

Extract only main oline headline for NYT.

In [ ]:
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return val  # Return the original value if it cannot be parsed

full_year_nyt["headline"] = full_year_nyt["headline"].apply(safe_literal_eval)

In [ ]:
full_year_nyt["headline"] = full_year_nyt["headline"].apply(lambda x: x["main"])

Extract one image url for NYT.

In [ ]:
def extract_first_url(multimedia_list):
    try:
        if isinstance(multimedia_list, list) and len(multimedia_list) > 0:
            return multimedia_list[0].get("url", None)
    except (ValueError, SyntaxError):
        return None
    return None

In [ ]:
full_year_nyt["multimedia"] = full_year_nyt["multimedia"].apply(safe_literal_eval)

In [ ]:
full_year_nyt["multimedia"] = full_year_nyt["multimedia"].apply(extract_first_url)

In [ ]:
full_year_nyt["multimedia"] = full_year_nyt["multimedia"].apply(lambda x: f"https://www.nytimes.com/{x}")

Convert the Zeit source field to extract newspaper name.

In [ ]:
full_year_zeit["source"] = full_year_zeit["source"].apply(safe_literal_eval)

In [ ]:
full_year_zeit["source"] = full_year_zeit["source"].apply(lambda x: x["title"])

Shorten snippet for Zeit data

In [ ]:
full_year_zeit["snippet"] = full_year_zeit["snippet"].apply(lambda x: x.split('.')[0] + '.' if isinstance(x, str) else x)

Flatten keywords for NYT data

In [ ]:
full_year_nyt["keywords"] = full_year_nyt["keywords"].apply(safe_literal_eval)

In [ ]:
full_year_nyt["keywords"] = full_year_nyt["keywords"].apply(lambda x: [d["value"] for d in x if isinstance(d, dict)])

In [ ]:
full_year_nyt

Provide same order of columns

In [ ]:
full_year_zeit = full_year_zeit[["_id", "source", "headline", "web_url", "snippet", "multimedia", "pub_date", "byline", "document_type", "keywords"]]

In [ ]:
full_year_nyt = full_year_nyt[["_id", "source", "headline", "web_url", "snippet", "multimedia", "pub_date", "byline", "document_type", "keywords"]]

In [ ]:
# Define columns that may have dict or list types
# This prevents error when posting data to Supabase using psql
json_columns = ["keywords", "byline"]

# Convert them to JSON strings
for col in json_columns:
    full_year_zeit[col] = full_year_zeit[col].apply(json.dumps)
    full_year_nyt[col] = full_year_nyt[col].apply(json.dumps)

In [ ]:
full_year_zeit

In [ ]:
full_year_nyt

## Export data to local folder

Run to export data to files in a local folder.

In [ ]:
full_year_nyt.to_csv("../input-data/nyt-temp-data.csv", index=False)

In [ ]:
full_year_zeit.to_csv("../input-data/zeit-temp-data.csv", index=False)

## Post data to Supabase

Run to post the data to the [PN Supabase](https://supabase.com/dashboard/project/cvzcwlcdubsgduukqodr).

In [ ]:
# Load variables from .env into environment
load_dotenv()

# Retrieve values
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
dbname = os.getenv("dbname")

In [ ]:
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

In [ ]:
full_year_nyt.to_sql("nyt_articles", engine, if_exists="replace", index=False)

In [ ]:
full_year_zeit.to_sql("zeit_articles", engine, if_exists="replace", index=False)